In [1]:
import joblib
import numpy as np
import pandas as pd
import gc
import time
import os
import sys
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import  f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from optbinning import OptimalBinning
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
from functools import reduce
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import optuna
from optuna import Trial
from sklearn.metrics import f1_score
from optuna.samplers import TPESampler
sampler = TPESampler(seed=42)


(CVXPY) Jan 25 11:02:21 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 25 11:02:21 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


> Seed Setting 

In [2]:

pd.set_option('display.max_rows', 500)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [98]:
# read data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')
original_columns = test.columns.tolist()

# Base Data Processing 

> to int, float 

In [99]:
import re
def extract_years(s):
    if "unknown" in s.lower():
        return np.nan  # or return some specific value
    elif "<" in s:
        return 0.5
    elif "+" in s :
        return 12 
    else:
        return int(re.search(r'\d+', s).group())
    
train['근로기간']= train['근로기간'].apply(extract_years)
test['근로기간']= test['근로기간'].apply(extract_years)

train['대출기간'] = train['대출기간'].apply(lambda x : int(x.split()[0]))
test['대출기간'] = test['대출기간'].apply(lambda x : int(x.split()[0]))


> Translate 

In [100]:
grade_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6}
column_dict = {
    '대출금액': 'amount',
    '대출기간': 'term',
    '근로기간': 'workperiod',
    '주택소유상태': 'homeown',
    '연간소득': 'income',
    '부채_대비_소득_비율': 'detinratio',
    '총계좌수': 'account_cnt',
    '대출목적': 'purpose',
    '최근_2년간_연체_횟수': 'delay_cnt',
    '총상환원금': 'principal_paid',
    '총상환이자': 'interest_paid',
    '총연체금액': 'delinquent_amount',
    '연체계좌수': 'delinquent_accounts',
    '대출등급': 'grade'
}

train['대출등급'] = train['대출등급'].map(grade_dict)

train = train.rename(columns=column_dict)
test = test.rename(columns=column_dict)

> Log Transform 


In [101]:
for var in ['income','account_cnt','principal_paid','interest_paid','delinquent_amount']:
    train[var+'_log'] = np.log(train[var]+1)
    test[var+'_log'] = np.log(test[var]+1)

> ANY to other

In [102]:
median_train = train['workperiod'].median()
train['homeown'] = train['homeown'].replace('ANY', 'MORTGAGE')
train['workperiod'] = train['workperiod'].fillna(median_train)
test['workperiod'] = test['workperiod'].fillna(median_train)

In [103]:
train

,ID,amount,term,workperiod,homeown,income,detinratio,account_cnt,purpose,delay_cnt,principal_paid,interest_paid,delinquent_amount,delinquent_accounts,grade,income_log,account_cnt_log,principal_paid_log,interest_paid_log,delinquent_amount_log
0,TRAIN_00000,12480000,36,6.0,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,2,18.092177,2.772589,0.000000,0.000000,0.0
1,TRAIN_00001,14400000,60,12.0,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,1,18.689180,3.091042,12.830869,12.363337,0.0
2,TRAIN_00002,12000000,36,5.0,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,0,18.379859,2.708050,13.741482,11.931274,0.0
3,TRAIN_00003,14400000,36,8.0,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,2,18.698312,2.772589,12.694116,11.938905,0.0
4,TRAIN_00004,18000000,60,6.0,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,1,18.088503,2.995732,12.339471,11.911413,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36,12.0,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,2,19.162618,3.526361,13.789763,13.106577,0.0
96290,TRAIN_96290,28800000,60,12.0,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,4,18.698312,3.258097,13.277192,13.658956,0.0
96291,TRAIN_96291,14400000,36,1.0,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,0,18.246327,3.135494,14.213702,12.393535,0.0
96292,TRAIN_96292,15600000,36,5.0,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,3,18.010153,3.091042,14.136411,13.614712,0.0


# Additional Data Processing

In [104]:
# '원금대비이자' 컬럼의 평균값 계산
mean_value = train['interest_paid'].mean()/train['principal_paid'].mean()
m1,m2 = train['interest_paid'].mean(), train['principal_paid'].mean()
def make_cols(data):
    data['timerate'] = data['workperiod']-(data['term']//12)
    data['totalpaid'] = data['interest_paid']+data['principal_paid']
    data['amountperyear'] =data['amount']//data['term']
    # data['debt'] = data['income']*data['detinratio']

    data['interestpaid_peryear'] = data['interest_paid']/data['term']
    data['principal_paid_peryear'] = data['principal_paid']/data['term']

    data['prin_int_rate'] = np.where((data['interest_paid'] == 0) & (data['principal_paid'] == 0), m1/(m1+m2), data['interest_paid']/(data['interest_paid']+data['principal_paid']))
    data['payrate']=np.where((data['interest_paid']> data['principal_paid']),1,0)

    data['paidrate']= data['totalpaid']/data['amountperyear']
    
    
make_cols(train)
make_cols(test)

> Grouped Features

In [105]:
# Group Operation
for group in ['term', 'homeown', 'purpose' ]:
    for num_var in ['amount', 'income','detinratio','delay_cnt','principal_paid','interest_paid','delinquent_amount']:
      
        train,test= train.copy(), test.copy()
        
        for stat in ['mean', 'max']:
            stat_dict = train.groupby([group])[num_var].agg(stat).to_dict()
            
            train[num_var +'_'+group + '_'+ stat]= train[group].map(stat_dict)
            test[num_var +'_'+group + '_'+ stat]= test[group].map(stat_dict)
            
            train[num_var+'_minus_'+num_var+'_'+group+'_'+stat] = train[num_var] - train[num_var+'_'+group+'_'+stat]
            test[num_var+'_minus_'+num_var+'_'+group+'_'+stat] = test[num_var] - test[num_var+'_'+group+'_'+stat]
            
            train[num_var+'_divide_'+num_var+'_'+group+'_'+stat] = np.log(train[num_var]+0.00001) / np.log(train[num_var+'_'+group+'_'+stat]+0.00001)
            test[num_var+'_divide_'+num_var+'_'+group+'_'+stat] = np.log(test[num_var]+0.00001) / np.log(test[num_var+'_'+group+'_'+stat]+0.00001)

In [106]:
# Woe Features
for variable in ['amount','workperiod','income','detinratio','principal_paid','interest_paid']:

    optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

    x = train[variable].values
    y = train['grade']

    optb.fit(x, y)

    binning_table = optb.binning_table

    a1 = binning_table.build()
    
    train[variable+'_woe'] = pd.cut(train[variable],bins=optb.splits.tolist()).astype(str)
    test[variable+'_woe'] = pd.cut(test[variable],bins=optb.splits.tolist()).astype(str)

> Feature Clustering 

In [107]:
from sklearn.cluster import KMeans

# 클러스터링할 피처들을 지정합니다.
features = ['amount','workperiod','income','detinratio','delinquent_amount']

# 최적의 클러스터 수로 KMeans 모델을 생성하고 학습합니다.
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=800, n_init=25, random_state=42)
kmeans.fit(train[features])

# 학습 데이터에 클러스터 번호를 추가합니다.
train['clust_num'] = kmeans.predict(train[features])

# 동일한 모델을 사용해 테스트 데이터에도 클러스터 번호를 추가합니다.
test['clust_num'] = kmeans.predict(test[features])


> Encoding

In [108]:
#인코딩 처리하기 
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

test['purpose'] = test['purpose'].replace('결혼', '기타')

encoder = LabelEncoder() 
    
for column in ['purpose','homeown','amount_woe','workperiod_woe','income_woe','detinratio_woe','principal_paid_woe','interest_paid_woe']:
    # train 데이터를 기반으로 인코딩을 학습하고 적용
    encoder.fit(train[column])
    train[column] = encoder.transform(train[column])
    test[column] = encoder.transform(test[column])

train_x = train.drop(['ID','grade'], axis=1)
test_x = test.drop('ID', axis=1)
train_y = train['grade']


In [76]:
#인코딩 처리하기 
# from sklearn.preprocessing import OneHotEncoder

# test['purpose'] = test['purpose'].replace('결혼', '기타')
# # 원-핫 인코더 생성
# encoder = OneHotEncoder(sparse=False)


# # 훈련 세트를 기반으로 인코더 학습
# encoder.fit(train[['homeown', 'term']])

# # 학습된 인코더를 훈련 세트와 테스트 세트에 적용
# train_encoded = encoder.transform(train[['homeown', 'term']])
# test_encoded = encoder.transform(test[['homeown', 'term']])

# # 인코딩된 결과를 데이터프레임으로 변환
# feature_names = np.concatenate(encoder.categories_).tolist()
# train_encoded_df = pd.DataFrame(train_encoded, columns=feature_names)
# test_encoded_df = pd.DataFrame(test_encoded, columns=feature_names)

# # 원래의 데이터프레임에서 '주택소유상태', '대출기간' 칼럼을 제거하고, 인코딩된 데이터프레임을 합침
# train = pd.concat([train.drop(['homeown', 'term'], axis=1), train_encoded_df], axis=1)
# test = pd.concat([test.drop(['homeown', 'term'], axis=1), test_encoded_df], axis=1)




In [80]:
# woe에 대해서도 일단 적용 
def catboost_encoder_multiclass(X,X_t,y):
    y = y.astype(str)
    enc = ce.OneHotEncoder().fit(y)
    y_onehot = enc.transform(y)
    class_names = y_onehot.columns
    X_obj = X.select_dtypes('object')
    X_t_obj = X_t.select_dtypes('object')
    X = X.select_dtypes(exclude='object')
    X_t = X_t.select_dtypes(exclude='object') 
    for class_ in class_names:
        enc = ce.CatBoostEncoder()
        enc.fit(X_obj,y_onehot[class_])
        temp = enc.transform(X_obj)
        temp_t = enc.transform(X_t_obj)
        temp.columns = [str(x)+'_'+str(class_) for x in temp.columns]
        temp_t.columns = [str(x)+'_'+str(class_) for x in temp_t.columns]
        X = pd.concat([X,temp],axis=1)
        X_t = pd.concat([X_t,temp_t],axis=1)
      
    return X, X_t

train, test= catboost_encoder_multiclass(train_x,test_x,train_y)

In [110]:
# feature making
from itertools import chain, product 
candidate_var = ['amountperyear','workperiod','income','detinratio','account_cnt','principal_paid','interest_paid']
pairs = list(chain(product(candidate_var, candidate_var), product(candidate_var, candidate_var))) 
pairs = pd.Series([sorted([i,j]) for (i,j) in set(pairs) if i!=j]).drop_duplicates().reset_index(drop=True).tolist()
pairs = sorted(pairs)
pairs_to_remove = [['interest_paid', 'principal_paid']]
pairs = [pair for pair in pairs if pair not in pairs_to_remove]

epsilon = 0.001  # 분모에 더해줄 작은 값

for i in range(len(pairs)):
    train[pairs[i][0]+'M'+pairs[i][1]] = train[pairs[i][0]] * train[pairs[i][1]]
    test[pairs[i][0]+'M'+pairs[i][1]] = test[pairs[i][0]] * test[pairs[i][1]]
    
    # 분모에 작은 값을 더해서 나눗셈 수행
    train[pairs[i][0]+'D'+pairs[i][1]] = np.log(train[pairs[i][0]]+epsilon) / np.log(train[pairs[i][1]] + epsilon)
    test[pairs[i][0]+'D'+pairs[i][1]] = np.log(test[pairs[i][0]]+epsilon) / np.log(test[pairs[i][1]] + epsilon)


# Data Set Processing

In [111]:
# 'train' 데이터프레임의 칼럼 이름을 문자열로 변환
train.columns = train.columns.map(str)
test.columns = test.columns.map(str)

# x = train.copy() 
# y = train_y.copy()


x = train.drop(['ID','grade'], axis=1 )
y = train['grade']
test  = test.drop('ID', axis=1)


> Check Models

In [112]:
# 데이터를 학습용과 검증용으로 분리
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# 사용할 모델들을 리스트에 담습니다.
models = [
    
    
    ('bag',BaggingClassifier()), 
    ('Random Forest', RandomForestClassifier()),
    ('LGBM',LGBMClassifier()),
    ('DTC', DecisionTreeClassifier()),
    ('XGBoost', XGBClassifier())
]

# 각 모델별로 학습 및 검증을 진행합니다.
for name, model in models:
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    f1 = f1_score(y_val, predictions, average='macro')
    print(f'{name} Macro F1 Score: {f1}')

bag Macro F1 Score: 0.8923735844323558
Random Forest Macro F1 Score: 0.8456319936678821
LGBM Macro F1 Score: 0.827555992507711
DTC Macro F1 Score: 0.8756687823415749


KeyboardInterrupt: 

# Model Training 

In [56]:
# from sklearn.tree import DecisionTreeClassifier 
# X.columns = X.columns.astype(str)
# test.columns = test.columns.astype(str)

# dtc = DecisionTreeClassifier()
# dtc.fit(X, y) 
# pred = dtc.predict(test)


In [60]:
from catboost import CatBoostClassifier
# 모델 생성
cb = CatBoostClassifier(iterations=300)
# 모델 훈련
cb.fit(X, y)
pred =cb.predict(test)

Learning rate set to 0.276261
0:	learn: 1.6256466	total: 79.5ms	remaining: 23.8s
1:	learn: 1.4549492	total: 103ms	remaining: 15.4s
2:	learn: 1.3430167	total: 121ms	remaining: 11.9s
3:	learn: 1.2607975	total: 134ms	remaining: 9.9s
4:	learn: 1.1897276	total: 147ms	remaining: 8.66s
5:	learn: 1.1501099	total: 158ms	remaining: 7.76s
6:	learn: 1.1031402	total: 171ms	remaining: 7.14s
7:	learn: 1.0626442	total: 187ms	remaining: 6.82s
8:	learn: 1.0367071	total: 201ms	remaining: 6.51s
9:	learn: 1.0144061	total: 218ms	remaining: 6.31s
10:	learn: 0.9889421	total: 229ms	remaining: 6.03s
11:	learn: 0.9729865	total: 240ms	remaining: 5.75s
12:	learn: 0.9496319	total: 251ms	remaining: 5.54s
13:	learn: 0.9332819	total: 261ms	remaining: 5.33s
14:	learn: 0.9243086	total: 269ms	remaining: 5.12s
15:	learn: 0.9156491	total: 278ms	remaining: 4.93s
16:	learn: 0.9046035	total: 286ms	remaining: 4.76s
17:	learn: 0.8991372	total: 296ms	remaining: 4.63s
18:	learn: 0.8903321	total: 304ms	remaining: 4.5s
19:	learn: 0

In [33]:
# 모델 생성
lgbm = LGBMClassifier(n_estimators=10000,n_jobs=-1)
# 모델 훈련
lgbm.fit(X, y)
pred = lgbm.predict(test)

> Xgboost Gridsearch 

In [13]:
# XGBoost Grid
from sklearn.model_selection import GridSearchCV

xgb_cv = StratifiedKFold(n_splits=10, shuffle = True, random_state=42)
params = {
    # 상황 보면서 파라미터 조절하기 
    'n_estimators': [100,200,300],
    'learning_rate': [0.01,0.05,0.1,0.15],
    'max_depth':[3,4,5,6,7,8] , 
    # 'gamma': [0, 0.5, 1],
    # 'alpha': [0, 0.5, 1],
    # 'lambda': [0.5, 1, 5],
    # 'min_child_weight': [1,2,3]
}

xgb_grid = GridSearchCV(XGBClassifier(n_jobs=-1), params, n_jobs=-1, cv=xgb_cv, scoring="f1_macro")
xgb_grid.fit(x, y)

print('Best score:', xgb_grid.best_score_)
print('Best score:', xgb_grid.best_params_)

Best score: 0.8190030126557973
Best score: {'learning_rate': 0.15, 'max_depth': 8, 'n_estimators': 300}


> Xgboost Optuna 

In [ ]:
# XGBoost Optuna

#train set 분리하기


def macro_f1_score(preds, dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(preds, axis=1)
    f1 = f1_score(y_true, y_pred, average='macro')
    return 'macro_f1', 1-f1  # XGBoost는 손실을 최소화하기 때문에 1 - f1을 반환


def objective(trial, train_x, train_y, val_x, val_y):
    
    param = {
        'objective': 'multi:softmax',  # 목표를 다중 클래스 분류로 설정
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate',0.0005, 0.15),
        'lambda': trial.suggest_float('lambda', 1e-3, 0.1),
        'alpha': trial.suggest_float('alpha', 1e-3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
        'max_depth': trial.suggest_int('max_depth',2,10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        
        
    }
    model =XGBClassifier(**param)  
    model.fit(train_x, train_y)
    preds = model.predict(val_x)
    score = macro_f1_score(val_y, preds)
    
    return score
          
# study_xgb.optimize(objective, n_trials=5)
study =  optuna.create_study(study_name='Xgb', direction='maximize',sampler=TPESampler(seed=42) )
study.optimize(lambda trial: objective(trial,X_train,  y_train ,X_valid , y_valid ),n_trials=10, timeout = 30 )
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)


In [15]:
# Predict the labels of the test data
pred = xgb_grid.best_estimator_.predict(test)

print(pred)


[1 1 0 ... 3 2 0]


In [94]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# 모델 정의
lgbm = LGBMClassifier()
xgb = XGBClassifier()
dtc = DecisionTreeClassifier()
bag = BaggingClassifier()
# VotingClassifier 생성
voting_clf = VotingClassifier(
    estimators=[('lgbm', lgbm), ('xgb', xgb), ('dtc', dtc), ('bag', bag)],
    voting='hard'  # 'hard' voting 또는 'soft' voting 선택
)

# 모델 학습
voting_clf.fit(x, y)

# 예측
y_pred = voting_clf.predict(test)


# Data Postprocessing

In [95]:
pred=y_pred
sample_submission['대출등급'] = pred 

# 딕셔너리 뒤집기
inv_grade_dict = {v: k for k, v in grade_dict.items()}

# 'grade' 열을 다시 알파벳으로 변경
sample_submission['대출등급'] = sample_submission['대출등급'].map(inv_grade_dict)


In [96]:

sample_submission

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C


In [97]:
sample_submission.to_csv('../data/dewoijwed.csv', index= False)